In [12]:
import numpy as np
import pandas as pd
import torch

from pathlib import Path
from sklearn.preprocessing import MinMaxScaler

from models.StackedAutoEncoder import SimpleAutoEncoder, AutoEncoder, StackedAutoEncoder
from models.LSTM import LSTM

In [13]:
data = pd.read_excel(Path("model_data/RawData.xlsx"), "S&P500 Index Data")
features = list(data.columns[2:])
# data[features] = data[features] / data[features].shift(1)
data.dropna(inplace=True)
train = data.loc[(data['Ntime'] >= 20120101) & (data['Ntime'] < 20140101)]
val = data.loc[(data['Ntime'] >= 20140101) & (data['Ntime'] < 20140401)]
test = data.loc[(data['Ntime'] >= 20140401) & (data['Ntime'] < 20140701)]
data = data[features]
data['target'] = data['Close Price'].shift(-1)
data.head()

,Close Price,Open Price,High Price,Low Price,Volume,MACD,CCI,ATR,BOLL,EMA20,MA10,MTM6,MA5,MTM12,ROC,SMI,WVAD,US Dollar Index,Federal Fund Rate,target
0,1284.91,1276.69,1285.31,1260.68,584629.0,-25.545595,-127.422348,24.63,1347.154231,1327.453482,1307.927,-33.09,1289.682,-75.12,-5.523408,-0.039542,-3.772245e+07,72.34,2.11,1261.52
1,1261.52,1285.82,1292.17,1261.51,527609.0,-27.571986,-115.864929,30.66,1342.391538,1321.174102,1300.298,-52.77,1277.592,-98.62,-7.250724,-0.060551,-5.204278e+07,71.99,1.95,1262.90
2,1262.90,1262.96,1271.48,1252.01,324759.0,-28.735319,-115.171527,19.47,1337.470769,1315.624188,1292.305,-59.07,1273.542,-88.03,-6.516252,-0.041755,-5.556148e+07,72.73,1.92,1252.31
3,1252.31,1262.90,1273.95,1240.68,526542.0,-30.164080,-111.302302,33.27,1331.857308,1309.594265,1285.743,-26.07,1268.328,-85.50,-6.740987,-0.049841,-5.352552e+07,72.71,1.99,1273.70
4,1273.70,1251.84,1274.17,1242.84,603411.0,-29.233405,-82.732708,31.33,1326.985000,1306.175764,1281.313,-6.30,1267.068,-69.13,-3.356020,-0.030884,-3.909113e+07,72.96,1.97,1244.69


In [14]:
def split_data(data, lookback):
    """"""
    scaler = MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    windows = []
    for index in range(len(data) - lookback): 
        windows.append(data[index: index + lookback])
    windows = np.array(windows)
    x = windows[:, :, :-1]
    y = windows[:, :, -1]
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.float32)
    return x, y

x_train, y_train = split_data(train, lookback=20)
x_val, y_val = split_data(val, lookback=20)
x_test, y_test = split_data(test, lookback=20)
x_train.shape

torch.Size([482, 20, 20])

In [15]:
y_train.shape

torch.Size([482, 20])

In [11]:
np.save(Path("model_data/x_train.npy"), x_train)
np.save(Path("model_data/x_val.npy"), x_val)
np.save(Path("model_data/x_test.npy"), x_test)
np.save(Path("model_data/y_train.npy"), y_train)
np.save(Path("model_data/y_val.npy"), y_val)
np.save(Path("model_data/y_test.npy"), y_test)